# Исследование тарифных планов

Компания «Мегалайн» — федеральный оператор сотовой связи. Клиентам предлагают два тарифных плана: «Смарт» и «Ультра».<br>
Чтобы скорректировать рекламный бюджет, коммерческий департамент хочет понять, какой тариф приносит больше денег.
Необходимо сделать предварительный анализ тарифов на небольшой выборке клиентов.<br> В вашем распоряжении данные 500 пользователей «Мегалайна»: кто они, откуда, каким тарифом пользуются, сколько звонков и сообщений каждый отправил за 2018 год. Нужно проанализировать поведение клиентов и сделать вывод — какой тариф лучше.

**Описание тарифов**

Тариф «Смарт»
* Ежемесячная плата: 550 рублей
* Включено 500 минут разговора, 50 сообщений и 15 Гб интернет-трафика
* Стоимость услуг сверх тарифного пакета: 1. минута разговора: 3 рубля («Мегалайн» всегда округляет вверх значения минут и мегабайтов. Если пользователь проговорил всего 1 секунду, в тарифе засчитывается целая минута); 2. сообщение: 3 рубля; 3. 1 Гб интернет-трафика: 200 рублей.


Тариф «Ультра»
* Ежемесячная плата: 1950 рублей
* Включено 3000 минут разговора, 1000 сообщений и 30 Гб интернет-трафика
* Стоимость услуг сверх тарифного пакета: 1. минута разговора: 1 рубль; 2. сообщение: 1 рубль; 3. 1 Гб интернет-трафика: 150 рублей.

*Примечание:*

*«Мегалайн» всегда округляет секунды до минут, а мегабайты — до гигабайт. Каждый звонок округляется отдельно: даже если он длился всего 1 секунду, будет засчитан как 1 минута.*

*Для веб-трафика отдельные сессии не считаются. Вместо этого общая сумма за месяц округляется в бо́льшую сторону. Если абонент использует 1025 мегабайт в этом месяце, с него возьмут плату за 2 гигабайта.*

**Задание на исследование:**

1. Необходимо исследовать: поведение клиентов оператора, исходя из выборки. Сколько минут разговора, сколько сообщений и какой объём интернет-трафика требуется пользователям каждого тарифа в месяц.

2. Проверка гипотез:
* Cредняя выручка пользователей тарифов «Ультра» и «Смарт» различаются.
* Cредняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов.

**Этапы иссслледования:**
1. [Обзор данных, изучение общей информации](#start)
2. [Предобработка данных](#preprocessing)<br>
   2.1. [Приведение данные к нужным типам:](#change)<br>
   2.2. [Поиск и исправление ошибок:](#mistakes)<br>
   2.3. [Расчёты и добавление результатов в таблицы](#calculation)<br> 
3. [Исследовательский анализ данных](#investigation)<br>
4. [Проверка гипотез](#hypothesis)<br>
   4.1. [Cредняя выручка пользователей тарифов «Ультра» и «Смарт» различаются](#hypothesis_1)<br>
   4.2. [Cредняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов.](#hypothesis_2)<br>
5. [Общий вывод](#conclusion)<br>

<a id ="start"></a>
## Обзор данных, изучение общей информации

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats as st

In [3]:
df_users = pd.read_csv('/datasets/users.csv')

In [4]:
df_tariffs = pd.read_csv('/datasets/tariffs.csv')

In [5]:
df_messages = pd.read_csv('/datasets/messages.csv')

In [6]:
df_internet = pd.read_csv('/datasets/internet.csv')

In [7]:
df_calls = pd.read_csv('/datasets/calls.csv')

In [8]:
df_calls['duration'] = np.ceil(df_calls['duration'])

In [9]:
df_users.sample(5)

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff
318,1318,53,NaN,Курган,Марьяна,Анищенко,2018-03-22,ultra
100,1100,58,NaN,Санкт-Петербург,Руслан,Красильников,2018-07-02,smart
40,1040,23,NaN,Сочи,Юрий,Одинцов,2018-06-08,smart
178,1178,56,NaN,Москва,Наталья,Жарова,2018-05-15,smart
472,1472,53,NaN,Ярославль,Наида,Боброва,2018-01-18,smart


In [10]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
user_id       500 non-null int64
age           500 non-null int64
churn_date    38 non-null object
city          500 non-null object
first_name    500 non-null object
last_name     500 non-null object
reg_date      500 non-null object
tariff        500 non-null object
dtypes: int64(2), object(6)
memory usage: 31.4+ KB


In [11]:
df_tariffs

,messages_included,mb_per_month_included,minutes_included,rub_monthly_fee,rub_per_gb,rub_per_message,rub_per_minute,tariff_name
0,50,15360,500,550,200,3,3,smart
1,1000,30720,3000,1950,150,1,1,ultra


In [12]:
df_tariffs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 8 columns):
messages_included        2 non-null int64
mb_per_month_included    2 non-null int64
minutes_included         2 non-null int64
rub_monthly_fee          2 non-null int64
rub_per_gb               2 non-null int64
rub_per_message          2 non-null int64
rub_per_minute           2 non-null int64
tariff_name              2 non-null object
dtypes: int64(7), object(1)
memory usage: 256.0+ bytes


In [13]:
df_messages.sample(5)

,id,message_date,user_id
71599,1302_427,2018-12-26,1302
83140,1346_51,2018-10-28,1346
11619,1048_265,2018-11-11,1048
84889,1354_341,2018-10-27,1354
59099,1257_891,2018-09-26,1257


In [14]:
df_messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123036 entries, 0 to 123035
Data columns (total 3 columns):
id              123036 non-null object
message_date    123036 non-null object
user_id         123036 non-null int64
dtypes: int64(1), object(2)
memory usage: 2.8+ MB


In [15]:
df_internet.sample(5)

,Unnamed: 0,id,mb_used,session_date,user_id
23217,23217,1078_134,515.37,2018-10-17,1078
20470,20470,1068_220,0.00,2018-09-14,1068
109962,109962,1374_173,235.88,2018-06-03,1374
148029,148029,1495_231,903.53,2018-09-11,1495
72538,72538,1244_370,95.89,2018-09-21,1244


In [16]:
df_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149396 entries, 0 to 149395
Data columns (total 5 columns):
Unnamed: 0      149396 non-null int64
id              149396 non-null object
mb_used         149396 non-null float64
session_date    149396 non-null object
user_id         149396 non-null int64
dtypes: float64(1), int64(2), object(2)
memory usage: 5.7+ MB


In [17]:
df_calls.sample(5)

,id,call_date,duration,user_id
174562,1431_251,2018-11-06,7.0,1431
15871,1043_295,2018-10-27,9.0,1043
119387,1308_173,2018-10-12,1.0,1308
150664,1377_1511,2018-03-23,6.0,1377
179703,1449_183,2018-12-02,4.0,1449


In [18]:
df_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202607 entries, 0 to 202606
Data columns (total 4 columns):
id           202607 non-null object
call_date    202607 non-null object
duration     202607 non-null float64
user_id      202607 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 6.2+ MB


**Вывод:**

1. **df_users (инормация о пользователях):** Пропусков данных нет (пропуски в столбце churn_date означают что абонент еще пользуется тарифом). В столбцах churn_date и reg_date необходимо изменить тип данных на datetime.
* user_id — уникальный идентификатор пользователя
* first_name — имя пользователя
* last_name — фамилия пользователя
* age — возраст пользователя (годы)
* reg_date — дата подключения тарифа (день, месяц, год)
* churn_date — дата прекращения пользования тарифом (если значение пропущено, то тариф ещё действовал на момент выгрузки данных)
* city — город проживания пользователя
* tarif — название тарифного плана
2. **df_tariffs (информация о тарифах):** Пропусков данных нет, типы данных соответствуют значениям.
* tariff_name — название тарифа
* rub_monthly_fee — ежемесячная абонентская плата в рублях
* minutes_included — количество минут разговора в месяц, включённых в абонентскую плату
* messages_included — количество сообщений в месяц, включённых в абонентскую плату
* mb_per_month_included — объём интернет-трафика, включённого в абонентскую плату (в мегабайтах)
* rub_per_minute — стоимость минуты разговора сверх тарифного пакета (например, если в тарифе 100 минут разговора в месяц, то со 101 минуты будет взиматься плата)
* rub_per_message — стоимость отправки сообщения сверх тарифного пакета
* rub_per_gb — стоимость дополнительного гигабайта интернет-трафика сверх тарифного пакета (1 гигабайт = 1024 мегабайта)
3. **df_messages (информация о сообщениях):** Пропусков данных нет. В столбце message_date необходимо изменить тип данных на datetime.
* id — уникальный номер сообщения
* message_date — дата сообщения
* user_id — идентификатор пользователя, отправившего сообщение
4. **df_internet (информация об интернет-сессиях):** Пропусков данных нет. В столбце session_date необходимо изменить тип данных на datetime.
* id — уникальный номер сессии
* mb_used — объём потраченного за сессию интернет-трафика (в мегабайтах)
* session_date — дата интернет-сессии
* user_id — идентификатор пользователя
5. **df_calls (информация о звонках):** Пропусков данных нет. В столбце calls_date необходимо изменить тип данных на datetime.
* id — уникальный номер звонка
* call_date — дата звонка
* duration — длительность звонка в минутах
* user_id — идентификатор пользователя, сделавшего звонок


<a id ="proprcessing"></a>
## Предобработка данных

Необходимо выполнить следуюие действия:
1. Приведение данных к нужным типам (данные в столбцах churn_date, reg_date, message_date, call_date, session_date  необходимо привести к формату datetime)
2. Поиск и исправление ошибок - анализ столбцов на наличие ошибок и исправление при необходимости.

<a id ="change"></a>
### Приведение данные к нужным типам:

In [19]:
df_users['churn_date'] = pd.to_datetime(df_users['churn_date'], format='%Y-%m-%d')

In [20]:
df_users['reg_date'] = pd.to_datetime(df_users['reg_date'], format='%Y-%m-%d')

In [21]:
df_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 8 columns):
user_id       500 non-null int64
age           500 non-null int64
churn_date    38 non-null datetime64[ns]
city          500 non-null object
first_name    500 non-null object
last_name     500 non-null object
reg_date      500 non-null datetime64[ns]
tariff        500 non-null object
dtypes: datetime64[ns](2), int64(2), object(4)
memory usage: 31.4+ KB


In [22]:
df_messages['message_date'] = pd.to_datetime(df_messages['message_date'], format='%Y-%m-%d')

In [23]:
df_messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123036 entries, 0 to 123035
Data columns (total 3 columns):
id              123036 non-null object
message_date    123036 non-null datetime64[ns]
user_id         123036 non-null int64
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 2.8+ MB


In [24]:
df_calls['call_date'] = pd.to_datetime(df_calls['call_date'], format='%Y-%m-%d')

In [25]:
df_calls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 202607 entries, 0 to 202606
Data columns (total 4 columns):
id           202607 non-null object
call_date    202607 non-null datetime64[ns]
duration     202607 non-null float64
user_id      202607 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 6.2+ MB


In [26]:
df_internet['session_date'] = pd.to_datetime(df_internet['session_date'], format='%Y-%m-%d')

In [27]:
df_internet.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149396 entries, 0 to 149395
Data columns (total 5 columns):
Unnamed: 0      149396 non-null int64
id              149396 non-null object
mb_used         149396 non-null float64
session_date    149396 non-null datetime64[ns]
user_id         149396 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 5.7+ MB


**Вывод**

Форматы столбцов приведены к необходимым значениям

<a id ="mistakes"></a>
### Поиск и исправление ошибок:

**Поиск ошибок в таблице df_users**

In [28]:
df_users.sample(10)

,user_id,age,churn_date,city,first_name,last_name,reg_date,tariff
19,1019,63,2018-10-05,Томск,Игнатий,Круглов,2018-05-16,ultra
254,1254,75,2018-08-17,Москва,Александра,Сухарева,2018-01-04,ultra
387,1387,74,NaT,Краснодар,Габриель,Зуев,2018-12-21,smart
183,1183,20,NaT,Владивосток,Снежана,Озерова,2018-07-12,smart
396,1396,23,NaT,Пенза,Эльга,Савельева,2018-01-12,ultra
471,1471,35,NaT,Вологда,Элеонора,Гаврилова,2018-02-26,ultra
319,1319,70,NaT,Санкт-Петербург,Харитон,Парфенов,2018-08-24,smart
194,1194,70,NaT,Москва,Егор,Наумов,2018-12-03,smart
147,1147,63,NaT,Якутск,Владимир,Синицын,2018-07-09,smart
157,1157,27,NaT,Москва,Виктория,Белоусова,2018-02-23,smart


In [29]:
df_users.duplicated().sum()

0

In [30]:
df_users['user_id'].duplicated().sum()

0

In [31]:
df_users.describe().T

,count,mean,std,min,25%,50%,75%,max
user_id,500.0,1249.500,144.481833,1000.0,1124.75,1249.5,1374.25,1499.0
age,500.0,46.588,16.667630,18.0,32.00,46.0,62.00,75.0


In [32]:
df_users['city'].unique()

array(['Краснодар', 'Москва', 'Стерлитамак', 'Новокузнецк',
       'Набережные Челны', 'Ульяновск', 'Челябинск', 'Пермь', 'Тюмень',
       'Самара', 'Санкт-Петербург', 'Иваново', 'Чебоксары', 'Омск', 'Уфа',
       'Томск', 'Чита', 'Мурманск', 'Петрозаводск', 'Тверь',
       'Екатеринбург', 'Махачкала', 'Вологда', 'Череповец', 'Ярославль',
       'Сочи', 'Хабаровск', 'Ставрополь', 'Рязань', 'Улан-Удэ', 'Тула',
       'Саратов', 'Ростов-на-Дону', 'Казань', 'Иркутск', 'Курск',
       'Калуга', 'Якутск', 'Астрахань', 'Химки', 'Владикавказ',
       'Волгоград', 'Саранск', 'Ижевск', 'Новосибирск', 'Брянск',
       'Тольятти', 'Нижний Тагил', 'Орёл', 'Белгород', 'Воронеж',
       'Волжский', 'Курган', 'Барнаул', 'Красноярск', 'Архангельск',
       'Липецк', 'Кемерово', 'Киров', 'Сургут', 'Пенза', 'Магнитогорск',
       'Нижний Новгород', 'Кострома', 'Владивосток', 'Тамбов', 'Балашиха',
       'Севастополь', 'Калининград', 'Оренбург', 'Смоленск',
       'Нижневартовск', 'Владимир', 'Новороссий

In [33]:
df_users['tariff'].value_counts()

smart    351
ultra    149
Name: tariff, dtype: int64

 В столбце df_users нет явных ошибок

**Втаблице df_tariffs нет явных ошибок**

**Поиск ошибок в таблице df_messages**

In [34]:
df_messages.sample(10)

,id,message_date,user_id
101166,1414_110,2018-09-10,1414
73283,1310_152,2018-08-28,1310
92986,1382_447,2018-05-24,1382
20437,1089_440,2018-09-24,1089
40753,1174_98,2018-09-28,1174
6475,1028_130,2018-10-20,1028
10684,1045_263,2018-02-21,1045
75575,1319_164,2018-09-13,1319
27834,1119_357,2018-08-13,1119
65714,1282_146,2018-02-12,1282


In [35]:
df_messages.duplicated().sum()

0

 В столбце df_messages нет явных ошибок

**Поиск ошибок в таблице df_internet**

In [36]:
df_internet.sample(10)

,Unnamed: 0,id,mb_used,session_date,user_id
20891,20891,1071_8,291.76,2018-04-10,1071
21805,21805,1072_528,75.46,2018-03-30,1072
3967,3967,1012_240,344.37,2018-09-16,1012
6396,6396,1021_40,660.86,2018-08-15,1021
73119,73119,1247_157,308.52,2018-09-10,1247
116010,116010,1391_143,848.65,2018-08-09,1391
131132,131132,1436_98,30.39,2018-10-25,1436
127662,127662,1428_100,358.91,2018-11-26,1428
59523,59523,1201_59,700.72,2018-08-13,1201
33528,33528,1118_145,490.54,2018-03-13,1118


In [37]:
df_internet.duplicated().sum()

0

В столбце df_internet нет явных ошибок

**Поиск ошибок в таблице df_calls**

In [38]:
df_calls.sample(10)

,id,call_date,duration,user_id
122477,1316_98,2018-12-24,8.0,1316
93478,1235_46,2018-10-23,11.0,1235
118087,1303_355,2018-09-13,11.0,1303
184282,1457_254,2018-12-24,15.0,1457
171149,1423_120,2018-06-12,7.0,1423
59630,1149_148,2018-07-08,15.0,1149
140109,1357_67,2018-12-03,2.0,1357
175294,1433_193,2018-03-02,1.0,1433
135,1000_135,2018-10-25,1.0,1000
146493,1372_671,2018-02-16,10.0,1372


In [39]:
df_calls.duplicated().sum()

0

В столбце df_calls нет явных ошибок

<a id ="calculation"></a>
### Расчёты и добавление результатов в таблицы:

**Количество сделанных звонков и израсходованных минут разговора по месяцам**

In [40]:
df_calls['month']= df_calls['call_date'].dt.month

Округление значений израсходованных минут > 0 до целого

In [41]:
df_calls['duration_round'] = np.ceil(df_calls[df_calls['duration'] > 0.0001]['duration']).astype('int64')

Звонки со значением 0 - пропущенные звонки, оставляем как 0

In [42]:
df_calls['duration_round'] = df_calls['duration_round'].fillna(0)

In [43]:
df_calls['duration_round'] = df_calls['duration_round'].astype('int64')

In [44]:
df_calls.sample(5)

,id,call_date,duration,user_id,month,duration_round
27447,1067_357,2018-05-17,11.0,1067,5,11
192227,1472_40,2018-05-13,1.0,1472,5,1
72425,1180_103,2018-12-30,7.0,1180,12,7
90362,1225_622,2018-08-30,10.0,1225,8,10
19866,1054_199,2018-11-13,5.0,1054,11,5


In [45]:
df_user_calls_month = df_calls.groupby(['user_id', 'month'])['duration_round'].agg(['sum', 'count']).reset_index().rename(columns = {'sum':'duration_month', 'count':'calles_month'})

In [47]:
# код ревьюреа
df_calls.groupby(['user_id', 'month']).agg(duration_month = ('duration_round', 'sum'),
                                        calles_month = ('duration_round', 'count')).reset_index()

,user_id,month,duration_month,calles_month
0,1000,5,159,22
1,1000,6,172,43
2,1000,7,340,47
3,1000,8,408,52
4,1000,9,466,58
...,...,...,...,...
3169,1498,10,247,41
3170,1499,9,70,9
3171,1499,10,449,68
3172,1499,11,612,74


In [ ]:
#df_user_calls_month.columns = ['user_id', 'month', 'duration_month', 'calles_month' ]

In [ ]:
df_user_calls_month.sample(5)

In [ ]:
df_user_calls_month.info()

**Количество отправленных сообщений по месяцам**

In [ ]:
df_messages['month']= df_messages['message_date'].dt.month

In [ ]:
df_user_messages_month = df_messages.groupby(['user_id', 'month'])['id'].agg('count').reset_index().rename(columns = {'id':'messages_month'})

In [ ]:
#df_user_messages_month.columns = ['user_id', 'month', 'messages_month']

In [ ]:
df_user_messages_month.sample(5)

In [ ]:
df_user_messages_month.info()

**Объем израсходованного интернет-трафика по месяцам**

In [ ]:
df_internet['month']= df_internet['session_date'].dt.month

In [ ]:
df_user_internet_month = df_internet.groupby(['user_id', 'month'])['mb_used'].agg('sum').reset_index().rename(columns = {'mb_used':'mb_used_month'})

In [ ]:
#df_user_internet_month.columns = ['user_id', 'month', 'mb_used_month']

In [ ]:
df_user_internet_month['gb_used_month'] = np.ceil(df_user_internet_month['mb_used_month']/1024).astype('int64')

In [ ]:
df_user_internet_month.sample(5)

In [ ]:
df_user_internet_month.info()

**Формирование итоговой таблицы:**

In [ ]:
df = df_user_messages_month.merge(df_user_calls_month, how='outer', on=['user_id', 'month'])

In [ ]:
df = df.merge(df_user_internet_month, how='outer', on=['user_id', 'month'])

In [ ]:
df = df_users.merge(df, how='outer', on='user_id')


In [ ]:
df.info()

Заменим нулевые NaN значения в столбцах month, messages_month, duration_month, calles_month, mb_used_month, gb_used_month на 0

In [ ]:
df.loc[:,'month':'gb_used_month'] = df.loc[:,'month':'gb_used_month'].fillna(0)

In [ ]:
df.loc[:,'month':'gb_used_month'] = df.loc[:,'month':'gb_used_month'].astype('int64')

In [ ]:
df.info()

In [ ]:
df.head(10)

**Помесячная выручка с каждого пользователя**

In [ ]:
df.loc[df['tariff'] == 'smart', 'revenue'] = (550 + np.clip(df['duration_month'] - 500, 0, None)*3 + np.clip(df['messages_month'] - 50, 0, None)*3 + np.clip(df['gb_used_month'] - 15, 0, None)*200)

In [ ]:
df.loc[df['tariff'] == 'ultra', 'revenue'] = (1950 + np.clip(df['duration_month'] - 3000, 0, None)*1 + np.clip(df['messages_month'] - 1000, 0, None)*1 + np.clip(df['gb_used_month'] - 30, 0, None)*150)

In [ ]:
df.sample(15)

In [ ]:
df.describe().T

**Вывод**

Сформирована итоговая таблица df для последующего анализа данных. Добавлена следующая информация в разрезе каждого клиента (user_id) помесячно :
* month - месяц
* messages_month - колличество сообщений в месяц
* duration_month - колличество использованных минут разговора в месяц
* calles_month - колличество звонков в месяц
* gb_used_month - колличество потраченных Гб в месяц
* revenue - выручка за месяц

Данные подготовлены для дальнейшего анализа.

<a id ="investigation"></a>
## Исследовательский анализ данных:

Необходимо исследовать: поведение клиентов оператора, исходя из выборки. Сколько минут разговора, сколько сообщений и какой объём интернет-трафика требуется пользователям каждого тарифа в месяц? Посчитайте среднее количество, дисперсию и стандартное отклонение. Постройте гистограммы. Опишите распределения.

In [ ]:
columns = ['duration_month', 'messages_month', 'gb_used_month', 'revenue']
tariffs = ['smart', 'ultra']
variance=[]
standard_deviation=[]
mean=[]
median=[]

for tariff in tariffs:
    for column in columns:
        variance.append(np.var(df.query('tariff == @tariff')[column], ddof=1))
        standard_deviation.append(np.std(df.query('tariff == @tariff')[column], ddof=1))
        mean.append(df.query('tariff == @tariff')[column].mean())
        median.append(df.query('tariff == @tariff')[column].median())
        
result = pd.DataFrame(data=[variance, standard_deviation, mean, median], 
                 columns=['Кол-во минут (смарт)', 'Кол-во СМС (смарт)', 'Кол-во ГБ (смарт)','Выручка(смарт)',
                         'Кол-во минут (ультра)', 'Кол-во СМС (ультра)', 'Кол-во ГБ (ультра)', 'Выручка(ультра)'],
                index = ['Дисперсия', 'Стандартное отклонение', 'Среднее значение', 'Медиана' ] )

In [ ]:
result

In [ ]:
columns = ['duration_month', 'messages_month', 'gb_used_month', 'revenue']
tariffs = ['smart', 'ultra']
for tariff in tariffs:
    for column in columns:
        fig = plt.figure()
        ax_1 = fig.add_subplot(1, 2, 1)
        df.query('tariff == @tariff')[column].hist(density=True)
        ax_2 = fig.add_subplot(1, 2, 2)
        df.query('tariff == @tariff').boxplot(column=column)
        ax_1.set_title(f'Тариф: {tariff},  Параметр:  {column}')
        ax_2.set_title(f'Тариф: {tariff},  Параметр:  {column}')
        fig.set_figwidth(12)
        fig.set_figheight(7)
        plt.show

**Вывод:**

1. Тариф smart:
- Среднее количество минут в месяц - 418, распределение - нормальное, немного скошено влево
- Среднее количество смс в месяц - 33, распределение - сильно скошено влево
- Среднее количество Гб в месяц - 16, распределение - нормальное.

2. Тариф ultra:
- Среднее количество минут в месяц - 526, распределение - нормальное,  скошено влево
- Среднее количество смс в месяц - 49, распределение - сильно скошено влево
- Среднее количество Гб в месяц - 19, распределение - нормальное.

In [ ]:
fig = plt.figure()
ax_1 = fig.add_subplot(1, 3, 1)
df.query('tariff == "smart"')['duration_month'].hist(alpha=0.75)
df.query('tariff == "ultra"')['duration_month'].hist(alpha=0.75)
ax_2 = fig.add_subplot(1, 3, 2)
df.query('tariff == "smart"')['messages_month'].hist(alpha=0.75)
df.query('tariff == "ultra"')['messages_month'].hist(alpha=0.75)
ax_3 = fig.add_subplot(1, 3, 3)
df.query('tariff == "smart"')['gb_used_month'].hist(alpha=0.75)
df.query('tariff == "ultra"')['gb_used_month'].hist(alpha=0.75)        
ax_1.set_title('duration_month')
ax_2.set_title('messages_month')
ax_3.set_title('gb_used_month')
fig.set_figwidth(15)
fig.set_figheight(7)
plt.show

<a id ="hypothesis"></a>
## Проверка гипотез:

<a id ="hypothesis_1"></a>
### Cредняя выручка пользователей тарифов «Ультра» и «Смарт» различаются

* Ho - Cредняя выручка пользователей тарифов «Ультра» и «Смарт» одинаковые
* H1 - Cредняя выручка пользователей тарифов «Ультра» и «Смарт» различаются

для проверки гипотезы о равенстве среднего двух генеральных совокупностей по взятым из них выборкам используем метод scipy.stats.ttest_ind (array1, array2,equal_var)

alpha = .05 # критический уровень статистической значимости<br>
если p-value окажется меньше него - отвегнем гипотезу

In [ ]:
results = st.ttest_ind(
    df.query('tariff == "smart"')['revenue'], 
    df.query('tariff == "ultra"')['revenue'],
    equal_var = False)

In [ ]:
results.pvalue

**Вывод**

**Отвергаем** нулевую гипотезу в пользу "H1 - Cредняя выручка пользователей тарифов «Ультра» и «Смарт» различаются", т.к. значение p-value намного меньше критического уровня статистической значимости

<a id ="hypothesis_2"></a>
### Cредняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов.

* Ho - Cредняя выручка пользователей из Москвы не отличается от выручки пользователей из других регионов.
* H1 - Cредняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов.

для проверки гипотезы о равенстве среднего двух генеральных совокупностей по взятым из них выборкам используем метод scipy.stats.ttest_ind (array1, array2,equal_var)

alpha = .05 # критический уровень статистической значимости<br>
если p-value окажется меньше него - отвегнем гипотезу

In [ ]:
results = st.ttest_ind(
    df.query('city == "Москва"')['revenue'], 
    df.query('city != "Москва"')['revenue'],
    equal_var = False)

In [ ]:
results.pvalue

**Вывод**

p-value намного больше критического уровня статистической значимости, гипотезу "Ho - Cредняя выручка пользователей из Москвы не отличается от выручки пользователей из других регионов." - **не отвергаем**



<a id ="conclusion"></a>
## Общий вывод:

1. На тарифе Смарт абоненты в среднем расходуют 418 минут разговоров, 33 смс и 17 Гб интернета. Средняя выручка - 1290 руб. <br> На тарифе Ультра - 526 минут, 50 смс и 20 Гб. Средняя выручка - 2070 р.

2. Проверка гипотез:
* Cредняя выручка пользователей тарифов «Ультра» и «Смарт» различаются - **гипотеза подтверждена.**
* Cредняя выручка пользователей из Москвы отличается от выручки пользователей из других регионов - **гипотеза отброшена.**.